<div style="text-align:center;">
    <h1>🔬 Target Trial Emulation with Clustering Enhancement</h1>
    <h3>Assignment 1: Clustering Integration in Target Trial Emulation (TTE)</h3>
    <h4>Authors:</h4>
    <ul style="list-style:none;">
        <li>👤 Elgen Mar Arinasa</li>
        <li>👤 Shawn Jurgen Mayol</li>
    </ul>
    <hr>
</div>

## 📖 Introduction
Target Trial Emulation (TTE) is a methodological framework in epidemiology designed to reduce biases that arise in observational studies. It allows researchers to simulate randomized controlled trials (RCTs) using observational data. Traditional observational study designs often suffer from selection bias and confounding, leading to unreliable causal inferences.

This notebook aims to **replicate the Target Trial Emulation (TTE) process** from the `TrialEmulation` R package in **Python**, ensuring that the results match those obtained in R. Additionally, we will explore a **novel integration of clustering techniques** within the TTE framework to improve the robustness of the analysis.

## 🎯 Objectives
This notebook will accomplish the following tasks:
1. **Load and inspect** the provided dataset (`data_censored.csv`).
2. **Convert the original R-based TTE methodology to Python** while maintaining accuracy.
3. **Perform Target Trial Emulation (TTE) Analysis** following established frameworks.
4. **Introduce Clustering Methods into TTE (TTE-v2)**:
   - Implement **K-Means clustering** to identify treatment response patterns.
   - Implement **DBSCAN clustering** to detect hidden structures and potential outlier effects.
5. **Compare the performance of traditional TTE vs. TTE with Clustering**.
6. **Generate insights** from the results, discussing improvements and trade-offs.

## 📚 Import Required Libraries
Before we begin, we will import the necessary libraries for data processing, visualization, and machine learning.

```python
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score


## 📂 Data Loading and Initial Inspection
To begin our analysis, we load the dataset (`data_censored.csv`) and inspect its structure. This step allows us to understand the data types, check for missing values, and verify that the dataset is correctly formatted for further analysis.


In [31]:
# Import necessary library
import pandas as pd

# Load the dataset
df = pd.read_csv("data_censored.csv")

# Display dataset shape
print("📌 Dataset Shape:", df.shape)

# Show the first few rows
df.head()


📌 Dataset Shape: (725, 12)


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0


### 🔍 Data Inspection
We will now perform an initial exploration of the dataset by:
1. Checking for missing values.
2. Inspecting data types to ensure correct formatting.
3. Summarizing key statistics of numerical and categorical columns.


In [32]:
# Check for missing values
print("\n🔎 Missing Values:")
print(df.isnull().sum())

# Get dataset info (column names, data types, and non-null counts)
print("\n📜 Dataset Info:")
df.info()

# Display summary statistics
print("\n📊 Dataset Summary:")
df.describe(include="all")



🔎 Missing Values:
id           0
period       0
treatment    0
x1           0
x2           0
x3           0
x4           0
age          0
age_s        0
outcome      0
censored     0
eligible     0
dtype: int64

📜 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725 entries, 0 to 724
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         725 non-null    int64  
 1   period     725 non-null    int64  
 2   treatment  725 non-null    int64  
 3   x1         725 non-null    int64  
 4   x2         725 non-null    float64
 5   x3         725 non-null    int64  
 6   x4         725 non-null    float64
 7   age        725 non-null    int64  
 8   age_s      725 non-null    float64
 9   outcome    725 non-null    int64  
 10  censored   725 non-null    int64  
 11  eligible   725 non-null    int64  
dtypes: float64(3), int64(9)
memory usage: 68.1 KB

📊 Dataset Summary:


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible
count,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.000000,725.00000,725.000000
mean,49.278621,7.051034,0.467586,0.405517,-0.173552,0.486897,-0.274722,48.093793,1.091149,0.015172,0.08000,0.234483
std,28.119313,5.802351,0.499293,0.491331,0.997552,0.500173,1.008643,11.834472,0.986206,0.122323,0.27148,0.423968
min,1.000000,0.000000,0.000000,0.000000,-3.284355,0.000000,-3.003087,19.000000,-1.333333,0.000000,0.00000,0.000000
25%,23.000000,2.000000,0.000000,0.000000,-0.809344,0.000000,-0.861899,40.000000,0.416667,0.000000,0.00000,0.000000
50%,50.000000,6.000000,0.000000,0.000000,-0.163060,0.000000,-0.316594,49.000000,1.166667,0.000000,0.00000,0.000000
75%,73.000000,12.000000,1.000000,1.000000,0.494103,1.000000,0.299510,56.000000,1.750000,0.000000,0.00000,0.000000
max,99.000000,19.000000,1.000000,1.000000,3.907648,1.000000,2.048087,78.000000,3.583333,1.000000,1.00000,1.000000


### 📌 Insights from Initial Inspection
- The dataset contains **`X` rows and `Y` columns**.
- **Key columns include:**
  - `id` → Unique identifier for patients.
  - `period` → Time period in the observation.
  - `treatment` → Binary variable indicating treatment assignment (`1 = treated`, `0 = control`).
  - `outcome` → Binary outcome variable (`1 = event occurred`, `0 = no event`).
  - `censored` → Whether the observation is censored (`1 = censored`, `0 = not censored`).
  - `eligible` → Binary indicator for eligibility in the trial.


## 🛠️ Data Preprocessing
Before proceeding with Target Trial Emulation (TTE), we need to ensure that the dataset is clean and formatted correctly. This step includes:
1. Handling missing values.
2. Ensuring correct data types.
3. Encoding categorical variables (if necessary).
4. Verifying unique patient IDs and periods.


In [33]:
# Handling missing values: Check if we need to drop or impute any missing data
missing_values = df.isnull().sum()
print("\n🔍 Missing Values Before Handling:\n", missing_values)

# Drop rows with critical missing values (if necessary)
df = df.dropna(subset=['id', 'period', 'treatment', 'outcome', 'eligible'])

# Convert categorical binary columns to integers (if needed)
binary_cols = ["treatment", "outcome", "censored", "eligible"]
df[binary_cols] = df[binary_cols].astype(int)

# Ensure period and id are integers
df["id"] = df["id"].astype(int)
df["period"] = df["period"].astype(int)

# Verify changes
print("\n✅ Data Types After Conversion:")
print(df.dtypes)

# Ensure unique patients exist in the dataset
unique_patients = df["id"].nunique()
print(f"\n👥 Unique Patients in the Dataset: {unique_patients}")

# Verify that each patient has multiple periods
period_counts = df.groupby("id")["period"].nunique().describe()
print("\n📊 Distribution of Periods per Patient:")
print(period_counts)



🔍 Missing Values Before Handling:
 id           0
period       0
treatment    0
x1           0
x2           0
x3           0
x4           0
age          0
age_s        0
outcome      0
censored     0
eligible     0
dtype: int64

✅ Data Types After Conversion:
id             int32
period         int32
treatment      int32
x1             int64
x2           float64
x3             int64
x4           float64
age            int64
age_s        float64
outcome        int32
censored       int32
eligible       int32
dtype: object

👥 Unique Patients in the Dataset: 89

📊 Distribution of Periods per Patient:
count    89.000000
mean      8.146067
std       7.570203
min       1.000000
25%       1.000000
50%       6.000000
75%      16.000000
max      20.000000
Name: period, dtype: float64


### 📌 Insights from Data Preprocessing
- **Missing values handled:** Critical missing entries removed.
- **Data types standardized:** Converted key variables (`treatment`, `outcome`, `censored`, `eligible`) to integer format.
- **Unique patients verified:** The dataset contains `X` unique patients.
- **Periods per patient analyzed:** Each patient has multiple observations across time.


## 🔬 Implementing Target Trial Emulation (TTE)
Target Trial Emulation (TTE) is a framework that allows observational studies to mimic randomized controlled trials (RCTs). In this step, we:
1. Define the **Intention-to-Treat (ITT)** and **Per-Protocol (PP)** estimands.
2. Expand the dataset into a sequence of target trials.
3. Handle **censoring mechanisms** due to treatment switching and informative censoring.


In [34]:
# Import necessary libraries
import numpy as np

# Define function for Target Trial Emulation setup
def setup_tte(data):
    """
    Prepares the dataset for Target Trial Emulation (TTE) by:
    - Setting treatment assignment
    - Expanding time periods
    - Handling censoring
    """
    df = data.copy()

    # Create time-on-regime column (how long a patient stays on treatment)
    df["time_on_regime"] = df.groupby("id")["treatment"].cumsum()

    # Create assigned_treatment column (same as initial treatment for ITT)
    df["assigned_treatment"] = df.groupby("id")["treatment"].transform("first")

    # Handle censoring mechanism (patients who switch treatment)
    df["censored_at_switch"] = ((df["treatment"] != df["assigned_treatment"]) & (df["treatment"].shift(1) == df["assigned_treatment"])).astype(int)

    return df

# Apply TTE setup function
tte_data = setup_tte(df)

# Display the first few rows after TTE setup
tte_data.head()


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible,time_on_regime,assigned_treatment,censored_at_switch
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1,1,1,0
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0,2,1,0
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0,3,1,0
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0,4,1,0
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0,5,1,0


### 📌 Key Transformations for Target Trial Emulation
- **Time on Regime (`time_on_regime`)** → Tracks how long a patient remains on their assigned treatment.
- **Assigned Treatment (`assigned_treatment`)** → Fixed at the patient's first treatment assignment.
- **Censoring at Treatment Switch (`censored_at_switch`)** → Marks when a patient deviates from their initial treatment.


## 🛠️ Handling Informative Censoring
To adjust for biases caused by patient dropout or treatment switching, we calculate **Inverse Probability of Censoring Weights (IPCW)**.


In [35]:
from sklearn.linear_model import LogisticRegression

def calculate_ipcw(data):
    """
    Calculates Inverse Probability of Censoring Weights (IPCW) using logistic regression.
    """
    df = data.copy()

    # Define features for censoring model
    censoring_features = ["age", "x1", "x2"]

    # Fit logistic regression model for censoring probability
    model = LogisticRegression()
    model.fit(df[censoring_features], df["censored"])

    # Predict censoring probability
    df["ipcw"] = 1 / (model.predict_proba(df[censoring_features])[:, 1] + 1e-6)  # Avoid division by zero

    return df

# Apply IPCW calculation
tte_data = calculate_ipcw(tte_data)

# Display first few rows after IPCW calculation
tte_data.head()


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible,time_on_regime,assigned_treatment,censored_at_switch,ipcw
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1,1,1,0,5.542201
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0,2,1,0,10.729435
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0,3,1,0,8.722839
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0,4,1,0,7.477850
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0,5,1,0,12.799620


### 📌 Handling Censoring with IPCW
- **Logistic Regression** is used to estimate the probability of censoring.
- **Inverse Probability of Censoring Weights (IPCW)** are applied to adjust for selection bias.


## 🔄 Expanding the Dataset into a Sequence of Trials
To properly estimate causal effects, we expand the dataset by:
1. Creating a **sequence of trials** (one per patient per time period).
2. Adjusting for **time-dependent covariates**.
3. Ensuring the data structure mimics a **randomized controlled trial (RCT)**.


In [36]:
def expand_trials(data):
    """
    Expands the dataset into a sequence of trials by creating multiple observations per patient.
    """
    df = data.copy()

    # Create a trial period column (starts from 0 for each patient)
    df["trial_period"] = df.groupby("id").cumcount()

    # Follow-up time is equivalent to the period column
    df["followup_time"] = df["period"]

    return df

# Apply expansion function
expanded_data = expand_trials(tte_data)

# Display first few rows of expanded dataset
expanded_data.head()


,id,period,treatment,x1,x2,x3,x4,age,age_s,outcome,censored,eligible,time_on_regime,assigned_treatment,censored_at_switch,ipcw,trial_period,followup_time
0,1,0,1,1,1.146148,0,0.734203,36,0.083333,0,0,1,1,1,0,5.542201,0,0
1,1,1,1,1,0.002200,0,0.734203,37,0.166667,0,0,0,2,1,0,10.729435,1,1
2,1,2,1,0,-0.481762,0,0.734203,38,0.250000,0,0,0,3,1,0,8.722839,2,2
3,1,3,1,0,0.007872,0,0.734203,39,0.333333,0,0,0,4,1,0,7.477850,3,3
4,1,4,1,1,0.216054,0,0.734203,40,0.416667,0,0,0,5,1,0,12.799620,4,4


### 📌 Key Transformations for Trial Expansion
- **`trial_period`** → Assigns a sequential trial period per patient.
- **`followup_time`** → Represents the time since the start of observation.


## 📊 Fitting the Marginal Structural Model (MSM)
To estimate causal effects, we fit a **Marginal Structural Model (MSM)**, which models the effect of treatment on the outcome while adjusting for time-varying confounders.


In [37]:
from statsmodels.api import Logit, add_constant

def fit_msm(data):
    """
    Fits a Marginal Structural Model (MSM) using logistic regression.
    """
    df = data.copy()

    # Ensure outcome is binary
    df["outcome"] = df["outcome"].apply(lambda x: 1 if x > 0 else 0)

    # Define MSM features
    msm_features = ["assigned_treatment", "x2", "followup_time", "trial_period"]

    # Add quadratic terms
    df["followup_time_squared"] = df["followup_time"] ** 2
    df["trial_period_squared"] = df["trial_period"] ** 2

    # Drop highly correlated columns dynamically
    corr_matrix = df[msm_features + ["followup_time_squared", "trial_period_squared"]].corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > 0.99)]

    df = df.drop(columns=to_drop)
    print("🛠️ Dropped Highly Correlated Features:", to_drop)

    # Fit logistic regression model
    X = add_constant(df)
    y = df["outcome"]

    model = Logit(y, X).fit()

    return model

# Re-run the MSM model
msm_model = fit_msm(expanded_data)

# Display summary
msm_model.summary()


🛠️ Dropped Highly Correlated Features: ['trial_period', 'trial_period_squared']
         Current function value: inf
         Iterations: 35


C:\Users\Shawn\AppData\Roaming\Python\Python312\site-packages\statsmodels\discrete\discrete_model.py:2385: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
C:\Users\Shawn\AppData\Roaming\Python\Python312\site-packages\statsmodels\discrete\discrete_model.py:2443: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))


LinAlgError: Singular matrix

### 📌 Understanding the MSM Model
- **Logistic regression** is used to model survival probabilities.
- **`assigned_treatment`** is the key variable of interest (causal effect estimation).
- **Time variables (`followup_time`, `trial_period`)** account for time-related confounding.
- **Quadratic terms (`followup_time_squared`, `trial_period_squared`)** allow for non-linear relationships.
